#Тестовое GPN по SQL

# 1. 

Задача состоит в том, чтобы составить полную таблицу (представление), в которой были бы явно приведены правила окраски и маркировки каждой из известных по первой таблице моделей охранного оборудования

## Решение

```sql
SELECT equipment.part_id, 
            CASE WHEN equip_color.paint_color IS NULL THEN 
                        (SELECT paint_color FROM equip_color WHERE part_id IS NULL) 
                        ELSE equip_color.paint_color END AS paint_color, 
            CASE WHEN equip_color.decal_color IS NULL THEN 
                        (SELECT decal_color FROM equip_color WHERE part_id IS NULL) 
                        ELSE equip_color.decal_color END AS decal_color
FROM equipment
LEFT JOIN equip_color USING(part_id);
```

# 2. 

Дана таблица сотрудников компании, где имя, фамилия и отчество находятся в отдельных полях. Требуется найти наиболее часто повторяющееся имя или имена (если сразу несколько имён встречаются одинаковое число раз).

## Решение

```sql
WITH t AS 
        (SELECT first_name, COUNT(first_name) AS cnt_name 
        FROM employee 
        GROUP BY first_name)
SELECT *
FROM t
GROUP BY first_name
HAVING cnt_name = (SELECT MAX(cnt_name) FROM t)
ORDER BY first_name;
```

# 3. 

Представьте, что есть некая иерархическая структура должностей. На них можно назначать/увольнять сотрудников. Набор необходимых таблиц и полей представлен в файле «Tables».

Также, сотрудники могут отсутствовать: отпуск, болезнь, командировка и т.п. - для простоты пусть будет произвольный текст. Интервалы могут пересекаться. Необходимые поля/таблицы опять же, на ваше усмотрение. Напишите:

1. запрос, выводящий ближайшего подчиненного (заместителя). Если их несколько, взять того, кто дольше работает на своей должности.

2. запрос, выводящий для указанной должности ближайшего активного (работающего в офисе) начальника. Более сложный вариант: на указанную дату.

3. запрос, выводящий список всех доступных (не отсутствующих) подчиненных указанного начальника. Более сложный вариант: на указанную дату.

Будет плюсом, если в качестве дополнительного задания вы напишете:

5. запрос, подсчитывающий число рабочих дней в указанный промежуток [дата с, дата по] для всех сотрудников, работавших в этот период хотя бы один день.

6. и/или количество дней (можно календарных), разбитых по колонками "в офисе", "в отпуске", "в командировке", "болел" и т.д. (количество и названия заранее не известны - это данные из п.3-4)

## Решение

Пример общей таблицы:

In [ ]:
stat_emp

,current_date,id_emp,name_emp,cheif_id,start_work,end_work,id_stat,start_stat_date,end_stat_date,name_stat
0,2021-08-01,1,Александр,NaN,2021-01-01,NaT,1,2021-07-01,NaT,в офисе
1,2021-08-02,2,Владимир,1.0,2021-01-11,NaT,2,2021-07-04,2021-07-11,болен
2,2021-08-03,3,Ольга,2.0,2021-01-21,NaT,3,2021-07-07,2021-07-17,в отпуске
3,2021-08-04,4,Николай,NaN,2021-01-31,NaT,1,2021-07-10,NaT,в офисе
4,2021-08-05,5,Светлана,3.0,2021-02-10,NaT,2,2021-07-13,2021-07-20,болен
5,2021-08-06,6,Виктор,1.0,2021-02-20,NaT,3,2021-07-16,2021-07-26,в отпуске
6,2021-08-07,7,Дмитрий,1.0,2021-03-02,NaT,3,2021-07-19,2021-07-29,в отпуске
7,2021-08-08,8,Антон,2.0,2021-03-12,2021-05-21,3,2021-07-22,2021-08-01,в отпуске
8,2021-08-09,9,Валерий,15.0,2021-03-22,NaT,1,2021-07-25,NaT,в офисе
9,2021-08-10,10,Роман,1.0,2021-04-01,2021-06-10,2,2021-07-28,2021-08-04,болен


## 3.1 
Запрос, выводящий ближайшего подчиненного (заместителя). Если их несколько, взять того, кто дольше работает на своей должности.

``` sql
WITH t1 AS
          (SELECT id_emp, name_emp
          FROM stat_emp),

t2 AS 
          (SELECT s.id_emp, s.name_emp, t1.name_emp AS chief_name, 
                  strftime('%J', 
                  date('now')) - strftime('%J',date(start_work) AS work_day
           FROM stat_emp AS s
           LEFT JOIN t1 ON t1.id_emp=s.cheif_id
           WHERE end_work IS NULL)
SELECT *
FROM t2
WHERE chief_name NOT NULL
GROUP BY chief_name
HAVING MAX(work_day)
```

Результат:

In [ ]:
select(sql)

,id_emp,name_emp,chief_name,work_day
0,2,Владимир,Александр,220.0
1,3,Ольга,Владимир,210.0
2,14,Оксана,Николай,100.0
3,5,Светлана,Ольга,190.0
4,15,Тимур,Светлана,90.0
5,9,Валерий,Тимур,150.0


## 3.2
Запрос, выводящий для указанной должности ближайшего активного (работающего в офисе) начальника. Более сложный вариант: на указанную дату.

```sql
WITH t1 AS
          (SELECT id_emp, name_emp
           FROM stat_emp),

t2 AS 
          (SELECT s.id_emp, s.name_emp, t1.name_emp AS chief_name, 
                  strftime('%J',date('now')) - strftime('%J',
                  date(start_work)) AS work_day, s.name_stat
           FROM stat_emp AS s
           INNER JOIN t1 ON t1.id_emp=s.cheif_id
           WHERE end_work IS NULL)

SELECT *
FROM t2
WHERE chief_name NOT NULL AND name_stat='в офисе'
GROUP BY chief_name
HAVING MAX(work_day)
ORDER BY id_emp
```

Результат:

In [ ]:
select(sql)

,id_emp,name_emp,chief_name,work_day,name_stat
0,9,Валерий,Тимур,150.0,в офисе
1,13,Анна,Ольга,110.0,в офисе


## 3.3
Запрос, выводящий список всех доступных (не отсутствующих) подчиненных указанного начальника. Более сложный вариант: на указанную дату.

Нет решения

## 3.5
Запрос, подсчитывающий число рабочих дней в указанный промежуток [дата с, дата по] для всех сотрудников, работавших в этот период хотя бы один день.

```sql

SELECT name_emp,
julianday(date('now')) - julianday(start_work) -
    (CASE WHEN julianday(date('now')) = julianday(start_work) THEN 0
     ELSE (CAST((julianday(date('now')) - julianday(start_work)) / 7 AS INTEGER) * 2
         ) +
      CASE WHEN strftime('%w', start_work) <= strftime('%w', date('now')) THEN 2
           ELSE strftime('%w', start_work) = '6' END
     END) AS num_work_day
FROM stat_emp
WHERE start_work BETWEEN '2021-01-01' AND '2021-08-30'
GROUP BY name_emp
HAVING num_work_day > 1

```

Результат:

In [ ]:
select(sql)

,name_emp,num_work_day
0,Александр,166.0
1,Андрей,92.0
2,Анна,79.0
3,Антон,116.0
4,Валерий,106.0
5,Виктор,129.0
6,Владимир,156.0
7,Дмитрий,120.0
8,Ирина,84.0
9,Николай,142.0


## 3.6
И/или количество дней (можно календарных), разбитых по колонками "в офисе", "в отпуске", "в командировке", "болел" и т.д. (количество и названия заранее не известны - это данные из п.3-4)

```sql

WITH t AS (

SELECT name_emp, strftime('%J',date('now')) - strftime('%J',date(start_work)) AS work_day, 
julianday(date('now')) - julianday(start_work) -
    (CASE WHEN julianday(date('now')) = julianday(start_work) THEN 0
    ELSE (CAST((julianday(date('now')) - julianday(start_work)) / 7 AS INTEGER) * 2
         ) +
      CASE WHEN strftime('%w', start_work) <= strftime('%w', date('now')) THEN 2
           ELSE strftime('%w', start_work) = '6'
      END
    END) AS num_work_day_all, 
    name_stat,
    CASE WHEN (julianday(end_stat_date) - julianday(start_stat_date) -
    (CASE WHEN julianday(end_stat_date) = julianday(start_stat_date) THEN 0
    ELSE (CAST((julianday(end_stat_date) - julianday(start_stat_date)) / 7 AS INTEGER) * 2
         ) +
      CASE WHEN strftime('%w', start_stat_date) <= strftime('%w', end_stat_date) THEN 2
           ELSE strftime('%w', start_stat_date) = '6'
      END
    END)) IS NULL THEN 0 ELSE 
    julianday(end_stat_date) - julianday(start_stat_date) -
    (CASE WHEN julianday(end_stat_date) = julianday(start_stat_date) THEN 0
    ELSE (CAST((julianday(end_stat_date) - julianday(start_stat_date)) / 7 AS INTEGER) * 2
         ) +
      CASE WHEN strftime('%w', start_stat_date) <= strftime('%w', end_stat_date) THEN 2
           ELSE strftime('%w', start_stat_date) = '6'
      END
    END)
    
     END AS num_fail_day_all 
FROM stat_emp
)

SELECT name_emp, 
  CASE WHEN (SUM(CASE WHEN name_stat = 'в офисе' THEN (num_work_day_all-num_fail_day_all) END)) IS NULL THEN 0 
  ELSE SUM(CASE WHEN name_stat = 'в офисе' THEN (num_work_day_all-num_fail_day_all) END) END AS 'в офисе',
  CASE WHEN (SUM(CASE WHEN name_stat = 'болен' THEN num_fail_day_all END)) IS NULL THEN 0
  ELSE SUM(CASE WHEN name_stat = 'болен' THEN num_fail_day_all END) END AS 'болен',
  CASE WHEN (SUM(CASE WHEN name_stat = 'в отпуске' THEN num_fail_day_all END)) IS NULL THEN 0
  ELSE SUM(CASE WHEN name_stat = 'в отпуске' THEN num_fail_day_all END) END AS 'в отпуске'
FROM t
GROUP BY name_emp;

```

Результат:

In [ ]:
select(sql)

,name_emp,в офисе,болен,в отпуске
0,Александр,166.0,0.0,0.0
1,Андрей,0.0,0.0,7.0
2,Анна,79.0,0.0,0.0
3,Антон,0.0,0.0,8.0
4,Валерий,106.0,0.0,0.0
5,Виктор,0.0,0.0,8.0
6,Владимир,0.0,3.0,0.0
7,Дмитрий,0.0,0.0,6.0
8,Ирина,84.0,0.0,0.0
9,Николай,142.0,0.0,0.0
